<a href="https://colab.research.google.com/github/postak/colazione-con-adk/blob/main/2025_09_Partners_ADK_Learning_session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```
Copyright 2025 Google LLC.
SPDX-License-Identifier: Apache-2.0
```

In [ ]:
#@title Second Session
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 🚀 Welcome to the Second Session of our Journey! 🚀

Welcome! This notebook is your definitive guide to giving AI agents superpowers through **tool integration**. An agent's true power isn't just its language model; it's its ability to connect to and interact with the outside world. We will explore the entire spectrum of tool integration patterns available in the Google Agent Development Kit (ADK).

By the end of this adventure, you will master how to:

- ✅ **Use Built-in & Custom Function Tools**: The foundational patterns for giving an agent new skills.
- ✅ **Generate Tools from Specifications**: Automatically create toolsets from **OpenAPI** specs and connect to enterprise APIs in **Google Cloud API Hub**.
- ✅ **Connect to Live Tool Servers**: Use **MCP** to have your agent dynamically discover tools hosted anywhere on the web.
- ✅ **Integrate with Other Frameworks**: Seamlessly use tools from the **LangChain** ecosystem directly within your ADK agent.
- ✅ **Share State Between Tools**: Use `ToolContext` to enable complex, multi-step workflows where tools can pass information to each other.

Let's dive into the toolkit!

---
### 🎁 🛑 Important Prerequisite: Setup Your Environment! 🛑 🎁
-----------------------------------------------------------------------------

You will need a **Google AI API Key** to run this notebook.

👉 Follow the instructions [here](https://github.com/postak/colazione-con-adk/blob/main/Setting%20Up%20Your%20GCP%20Project%20%26%20Gemini%20API%20Key.pdf)


---
## Part 0: Setup & Authentication 🔑

Let's install all necessary libraries and configure your API key. This single setup will prepare you for every example in the notebook.

In [ ]:
# Install all the packages we'll need for this entire tutorial
# This includes the ADK, Google's AI library, and libraries for specific integrations
!pip install google-adk google-generativeai mcp requests nest-asyncio langchain-community tavily-python wikipedia -q

# --- Import all necessary libraries ---
import asyncio
import os
import json
import requests
import traceback
from getpass import getpass
from IPython.display import display, Markdown

# ADK and Tool imports
from google.adk.agents import Agent
from google.adk.tools import google_search, ToolContext
from google.adk.tools.agent_tool import AgentTool
from google.adk.tools.openapi_tool import OpenAPIToolset
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset
from google.adk.tools.mcp_tool.mcp_session_manager import SseServerParams
from google.adk.tools.langchain_tool import LangchainTool
# Note: APIHubToolset is shown for conceptual purposes
# from google.adk.tools.apihub_tool import APIHubToolset

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session

# Google AI imports
import google.generativeai as genai
from google.genai.types import Content, Part

# LangChain imports for Part 4
from langchain_community.tools import TavilySearchResults

# A helper to run async ADK code in Colab/Jupyter
import nest_asyncio
nest_asyncio.apply()

print("✅ All libraries are installed and ready to go!")

In [ ]:
google_api_key = getpass('Enter your Google API Key: ')
genai.configure(api_key=google_api_key)
os.environ['GOOGLE_API_KEY'] = google_api_key
print("✅ Google API Key configured.")

---

## Part 1: The Basics - Direct Tooling

These are the most fundamental ways to give an agent new skills.

### 1.1 Built-in Tools
The ADK comes with pre-packaged tools. `Google Search` is the perfect example, giving your agent immediate access to real-time information.

In [ ]:
# Define an agent and give it the built-in Google Search tool

# --- Agent Definition ---

def create_day_trip_agent():
    """Create the Spontaneous Day Trip Generator agent"""
    return Agent(
        name="day_trip_agent",
        model="gemini-2.5-flash",
        description="Agent specialized in generating spontaneous full-day itineraries based on mood, interests, and budget.",
        instruction="""
        You are the "Spontaneous Day Trip" Generator 🚗 - a specialized AI assistant that creates engaging full-day itineraries.

        Your Mission:
        Transform a simple mood or interest into a complete day-trip adventure with real-time details, while respecting a budget.

        Guidelines:
        1. **Budget-Aware**: Pay close attention to budget hints like 'cheap', 'affordable', or 'splurge'. Use Google Search to find activities (free museums, parks, paid attractions) that match the user's budget.
        2. **Full-Day Structure**: Create morning, afternoon, and evening activities.
        3. **Real-Time Focus**: Search for current operating hours and special events.
        4. **Mood Matching**: Align suggestions with the requested mood (adventurous, relaxing, artsy, etc.).

        RETURN itinerary in MARKDOWN FORMAT with clear time blocks and specific venue names.
        """,
        tools=[google_search]
    )

day_trip_agent = create_day_trip_agent()
print(f"🧞 Agent '{day_trip_agent.name}' is created and ready for adventure!")

In [ ]:
# --- A Helper Function to Run Our Agents ---
# We'll use this function throughout the notebook to make running queries easy.

async def run_day_trip_agent_query(agent: Agent, query: str, session: Session, user_id: str, is_router: bool = False):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if not is_router:
                # Let's see what the agent is thinking!
                print(f"EVENT: {str(event)[:300]} \n...\n...\n<output truncated>\n\n")
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    if not is_router:
     print("\n" + "-"*50)
     print("✅ Final Response:")
     display(Markdown(final_response))
     print("-"*50 + "\n")

    return final_response

# --- Initialize our Session Service ---
# This one service will manage all the different sessions in our notebook.
session_service = InMemorySessionService()
my_user_id = "adk_adventurer_001"

In [ ]:
# --- Let's test the Day Trip Genie! ---

async def run_day_trip_genie():
    # Create a new, single-use session for this query
    day_trip_session = await session_service.create_session(
        app_name=day_trip_agent.name,
        user_id=my_user_id
    )

    # Note the new budget constraint in the query!
    query = "Plan a relaxing and artsy day trip near Sunnyvale, CA. Keep it affordable!"
    print(f"🗣️ User Query: '{query}'")

    await run_day_trip_agent_query(day_trip_agent, query, day_trip_session, my_user_id)

await run_day_trip_genie()

### A Helper Function to Run Our Agents
To keep our code clean, we'll define a single helper function to manage running our agents.

In [ ]:
# Initialize a session service to manage conversations
session_service = InMemorySessionService()
my_user_id = "adk_toolkit_user"

async def run_agent_query(agent: Agent, query: str):
    """A reusable function to run a query against any agent."""
    session = await session_service.create_session(app_name=agent.name, user_id=my_user_id)
    print(f"\n🚀 Running query for agent: '{agent.name}'...")

    runner = Runner(agent=agent, session_service=session_service, app_name=agent.name)
    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=my_user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            # To see the agent's full thought process, uncomment the line below
            # print(f"EVENT: {event}")
            if event.is_final_response() and event.content.parts:
                final_response += event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"
        print("\n🔍 Full traceback:")
        print(traceback.format_exc())

    print("\n" + "-"*50)
    print("✅ Final Response:")
    display(Markdown(final_response))
    print("-"*50 + "\n")
    return final_response

print("✅ Agent query helper function defined.")

### 1.2 Custom Function Tools
This is the most common pattern: wrapping your own Python function into a tool. The function's **docstring** is crucial, as it tells the LLM what the tool does and when to use it. Here, we'll create a tool to fetch live weather data from the public U.S. National Weather Service API.

In [ ]:
# --- Tool Definition: A function that calls a live public API ---
# A simple lookup to avoid needing a separate geocoding API for this example
LOCATION_COORDINATES = {
    "sunnyvale": "37.3688,-122.0363",
    "san francisco": "37.7749,-122.4194",
    "lake tahoe": "39.0968,-120.0324"
}

def get_live_weather_forecast(location: str) -> dict:
    """Gets the current, real-time weather forecast for a specified location in the US.

    Args:
        location: The city name, e.g., "San Francisco".

    Returns:
        A dictionary containing the temperature and a detailed forecast.
    """
    print(f"\n🛠️ TOOL CALLED: get_live_weather_forecast(location='{location}')\n")

    # Find coordinates for the location
    normalized_location = location.lower()
    coords_str = None
    for key, val in LOCATION_COORDINATES.items():
        if key in normalized_location:
            coords_str = val
            break
    if not coords_str:
        return {"status": "error", "message": f"I don't have coordinates for {location}."}

    try:
        # NWS API requires 2 steps: 1. Get the forecast URL from the coordinates.
        points_url = f"https://api.weather.gov/points/{coords_str}"
        headers = {"User-Agent": "ADK Example Notebook"}
        points_response = requests.get(points_url, headers=headers)
        points_response.raise_for_status() # Raise an exception for bad status codes
        forecast_url = points_response.json()['properties']['forecast']

        # 2. Get the actual forecast from the URL.
        forecast_response = requests.get(forecast_url, headers=headers)
        forecast_response.raise_for_status()

        # Extract the relevant forecast details
        current_period = forecast_response.json()['properties']['periods'][0]
        return {
            "status": "success",
            "temperature": f"{current_period['temperature']}°{current_period['temperatureUnit']}",
            "forecast": current_period['detailedForecast']
        }
    except requests.exceptions.RequestException as e:
        return {"status": "error", "message": f"API request failed: {e}"}

# --- Agent Definition: An agent that USES the new tool ---
weather_agent = Agent(
    name="weather_aware_planner",
    model="gemini-2.5-flash",
    description="A trip planner that checks the real-time weather before making suggestions.",
    instruction="You are a cautious trip planner. Before suggesting any outdoor activities, you MUST use the `get_live_weather_forecast` tool to check conditions. Incorporate the live weather details into your recommendation.",
    tools=[get_live_weather_forecast]
)

print(f"🌦️ Agent '{weather_agent.name}' is created and can now call a live weather API!")

# --- Let's test the Weather-Aware Planner ---
await run_agent_query(weather_agent, "I want to go hiking near Lake Tahoe tomorrow, what's the weather like?")

---
## Part 2: Specification-Driven Tools

Instead of writing function by function, you can provide the agent with a formal specification, and it will generate the necessary tools automatically.

### 2.1 From an OpenAPI Spec (`OpenAPIToolset`)
If you have an existing API with an OpenAPI specification, you can instantly turn it into a toolset for your agent. This is incredibly powerful for integrating with existing microservices.

In [ ]:
# A simple OpenAPI 3.0 spec for a Pet Store as a string
pet_store_spec = """
{
  "openapi": "3.0.0",
  "info": {
    "title": "Simple Pet Store API (Mock)",
    "version": "1.0.1",
    "description": "An API to manage pets in a store, using httpbin for responses."
  },
  "servers": [
    {
      "url": "https://httpbin.org",
      "description": "Mock server (httpbin.org)"
    }
  ],
  "paths": {
    "/get": {
      "get": {
        "summary": "List all pets (Simulated)",
        "operationId": "listPets",
        "description": "Simulates returning a list of pets. Uses httpbin's /get endpoint which echoes query parameters.",
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "Maximum number of pets to return",
            "required": false,
            "schema": { "type": "integer", "format": "int32" }
          },
          {
             "name": "status",
             "in": "query",
             "description": "Filter pets by status",
             "required": false,
             "schema": { "type": "string", "enum": ["available", "pending", "sold"] }
          }
        ],
        "responses": {
          "200": {
            "description": "A list of pets (echoed query params).",
            "content": { "application/json": { "schema": { "type": "object" } } }
          }
        }
      }
    },
    "/post": {
      "post": {
        "summary": "Create a pet (Simulated)",
        "operationId": "createPet",
        "description": "Simulates adding a new pet. Uses httpbin's /post endpoint which echoes the request body.",
        "requestBody": {
          "description": "Pet object to add",
          "required": true,
          "content": {
            "application/json": {
              "schema": {
                "type": "object",
                "required": ["name"],
                "properties": {
                  "name": {"type": "string", "description": "Name of the pet"},
                  "tag": {"type": "string", "description": "Optional tag for the pet"}
                }
              }
            }
          }
        },
        "responses": {
          "201": {
            "description": "Pet created successfully (echoed request body).",
            "content": { "application/json": { "schema": { "type": "object" } } }
          }
        }
      }
    },
    "/get/{petId}": {
      "get": {
        "summary": "Info for a specific pet (Simulated)",
        "operationId": "showPetById",
        "description": "Simulates returning info for a pet ID. Uses httpbin's /get endpoint.",
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "description": "This is actually passed as a query param to httpbin /get",
            "required": true,
            "schema": { "type": "integer", "format": "int64" }
          }
        ],
        "responses": {
          "200": {
            "description": "Information about the pet (echoed query params)",
            "content": { "application/json": { "schema": { "type": "object" } } }
          },
          "404": { "description": "Pet not found (simulated)" }
        }
      }
    }
  }
}
"""

# 1. Create the toolset from the OpenAPI spec string
pet_store_toolset = OpenAPIToolset(
    spec_str=pet_store_spec, spec_str_type='json'
)

# 2. Create an agent that uses this toolset
pet_store_agent = Agent(
    name="pet_store_agent",
    model="gemini-2.5-flash",
    instruction="You are a Pet Store assistant. Use your tools to find information about pets.",
    tools=[pet_store_toolset]
)

# 3. Run the agent
# The agent will see the `getPetById` tool and know how to call the API.
await run_agent_query(pet_store_agent, "Show me the pets available.")

await run_agent_query(pet_store_agent, "Please add a new dog named 'Snoopy'.")

await run_agent_query(pet_store_agent, "Get info for pet with ID 123.")

### 2.2 From a Public Server (`MCPToolset`)
**MCP** is a standard that lets agents discover tools from a remote server. We can use `MCPToolset` to connect to a public server hosting tools for the MDN Web Docs.

In [ ]:
# The URL for the public server hosting MDN documentation tools
MCP_SERVER_URL = "https://gitmcp.io/mdn/content"

# This is an async function because MCPToolset needs to connect to the server
async def create_mdn_agent():
    mcp_toolset = MCPToolset(
        connection_params=SseServerParams(url=MCP_SERVER_URL)
    )
    return Agent(
        name="mdn_docs_assistant",
        model="gemini-2.5-flash",
        tools=[mcp_toolset],
        instruction="You are a web dev expert with access to MDN docs. Use your tools to answer questions."
    )

# Create and run the agent
mdn_agent = await create_mdn_agent()
await run_agent_query(mdn_agent, "What is the CSS `box-sizing` property?")

---
## Part 3: Interoperability

You don't have to build everything from scratch. The ADK is designed to work with other popular AI frameworks.

### 3.1 Using LangChain Tools (`LangchainTool`)
If you have existing tools built with LangChain, you can wrap them using `LangchainTool` and use them directly in your ADK agent. Here, we'll wrap LangChain's `WikipediaAPIWrapper` tool.

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from google.adk.tools.langchain_tool import LangchainTool

# 1. Instantiate the original LangChain tool
# This tool queries the public Wikipedia API.
wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# 2. Wrap it for ADK using LangchainTool
adk_wrapped_wiki_tool = LangchainTool(tool=wikipedia_tool)

# 3. Use the wrapped tool in your ADK agent
wiki_agent = Agent(
    name="wiki_research_agent",
    model="gemini-2.5-flash",
    instruction="You are a research assistant. Use the Wikipedia tool to answer the user's question.",
    tools=[adk_wrapped_wiki_tool]
)

# 4. Run the agent
await run_agent_query(wiki_agent, "What is the history of the Slinky toy?")

## Part 4: Advanced Composition
Now let's combine these concepts to build more sophisticated systems.



### 4.1 Sharing State Between Tools (ToolContext)
What if one tool needs information gathered by another? ToolContext is a special object that can be passed to your tool functions, allowing them to read and write to a shared state dictionary within a single conversational turn.


In [ ]:
def set_user_preference(preference: str, value: str, tool_context: ToolContext):
    """Saves a user's preference for this session.

    Args:
        preference: The name of the preference (e.g., 'theme').
        value: The value of the preference (e.g., 'dark').
    """
    # The state is a simple dictionary unique to this turn
    tool_context.state[preference] = value
    print(f"\n🛠️ TOOL CALLED: Set preference '{preference}' to '{value}'\n")
    return {"status": "success", "message": f"Preference saved."}

def get_user_preference(preference: str, tool_context: ToolContext):
    """Gets a previously saved user preference.

    Args:
        preference: The name of the preference to retrieve.
    """
    value = tool_context.state.get(preference, "not set")
    print(f"\n🛠️ TOOL CALLED: Retrieved preference '{preference}', value is '{value}'\n")
    return {"preference": preference, "value": value}


stateful_agent = Agent(
    name="stateful_agent",
    model="gemini-2.5-flash",
    instruction="First, save the user's preference. Then, retrieve it and confirm it back to them.",
    tools=[set_user_preference, get_user_preference]
)

await run_agent_query(stateful_agent, "Please set my theme preference to 'dark mode'.")